In [ ]:
from markitdown import MarkItDown
from PyPDF2 import PdfReader,PdfWriter

pdf_path = "doctests/Apsis Solutions Ltd  and IUTCS Agreement.pdf"
reader = PdfReader(pdf_path)
md = MarkItDown()

pages_markdown = []
for i, page in enumerate(reader.pages, start=1):
    # Extract one page into a temporary PDF file
    writer = PdfWriter()
    writer.add_page(page)
    tmp_pdf = f"page_{i}.pdf"
    with open(tmp_pdf, "wb") as f:
        writer.write(f)

    # Convert that single page
    result = md.convert(tmp_pdf)
    pages_markdown.append((i, result.markdown))

In [ ]:
result=md.convert("/home/ansary/work/apsis/DocSignerNML/document_ai_verification/tests/page_1.pdf")
result

In [ ]:
result.title

In [ ]:
result.markdown

In [ ]:
pages_markdown[0][1]

In [ ]:
result=md.convert("/home/ansary/work/apsis/DocSignerNML/document_ai_verification/tests/output.pdf")
result

In [ ]:
result.markdown

In [ ]:
from PIL import Image

# Open image
image = Image.open("/home/ansary/work/apsis/DocSignerNML/document_ai_verification/sample_document.png")

# Convert to RGB (important for JPG/PNG)
if image.mode == "RGBA":
    image = image.convert("RGB")

# Save as PDF
image.save("output.pdf", "PDF")